**Further train DeepCell Models with K's Data and Make Predictions**<br>
Starting from model trained with tn nuclear data, further train the model with resized K's images that is cut into 25 512x512 patches, one image at a time.
200 epochs, semanticdatagenerator, resized<br>
Optimize data augmentation.

In [1]:
starting_model_path = 'tn1.0_nuclear_20221102.h5' # the model trained with nucleus data from the Tissuenet V1.0 dataset
epochs=200 # 20 sec per step 

In [2]:
import syotil

import numpy as np
from skimage import io
from matplotlib import pyplot as plt
%matplotlib inline
from timeit import default_timer
import os
from skimage import img_as_ubyte

import deepcell
from deepcell import image_generators
from deepcell.utils.train_utils import rate_scheduler, get_callbacks, count_gpus
from deepcell_toolbox.utils import resize, tile_image, untile_image
from deepcell_toolbox.deep_watershed import deep_watershed
from deepcell.losses import weighted_categorical_crossentropy
from deepcell.model_zoo.panopticnet import PanopticNet

import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import MSE

print(tf.__version__)
print(deepcell.__version__)

2.7.1
0.11.1


In [3]:
INPUT_PATH="images/training7/"
import glob
FILENAMES = glob.glob(INPUT_PATH+"*_img.png")
print(len(FILENAMES))

imgs = [io.imread(CURR_IM_NAME)[:,:,0] for CURR_IM_NAME in FILENAMES]
X_train = tf.stack(imgs)
X_train = np.expand_dims(X_train, axis=-1)
print(X_train.shape)

masks = [io.imread(CURR_IM_NAME.replace("img","masks")) for CURR_IM_NAME in FILENAMES]
y_train = tf.stack(masks)
y_train = np.expand_dims(y_train, axis=-1)
print(y_train.shape)

# resize image
mpp_select=1.3 # based on DeepCell_tn_nuclear.ipynb, the best value is 1.2, but 1.3 makes scale factor an integer
shape = X_train.shape
scale_factor = mpp_select / 0.65
print(scale_factor)
new_shape = (int(shape[1] * scale_factor),
             int(shape[2] * scale_factor))

X_train_resized = resize(X_train, new_shape, data_format='channels_last')
y_train_resized = resize(y_train, new_shape, data_format='channels_last', labeled_image=True)
print(X_train_resized.shape)
print(y_train_resized.shape)

# cut each resized image into 5x5 patches of size 512x512 
print(X_train_resized.shape)
print(y_train_resized.shape)

x_starts=[0,400,400*2,400*3,X_train_resized.shape[1]-512]
x_ends=[x+512 for x in x_starts]
print(x_starts)
print(x_ends)

y_starts=[0,512,512*2,512*3,X_train_resized.shape[2]-512]
y_ends=[x+512 for x in y_starts]
print(y_starts)
print(y_ends)

sub_X=tf.concat([tf.concat([X_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :] for i in range(len(x_starts)) ], axis=0) for j in range(len(y_starts))], axis=0)
sub_X = np.array(sub_X, dtype=np.uint8) # tf.stack changes the dtype 
print(sub_X.shape)
sub_y=tf.concat([tf.concat([y_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :] for i in range(len(x_starts)) ], axis=0) for j in range(len(y_starts))], axis=0)
sub_y = np.array(sub_y, dtype=np.uint16) # tf.stack changes the dtype 
print(sub_y.shape)

# create dataloader
seed=0
min_objects = 5

transforms = ['inner-distance', 'outer-distance', 'fgbg']
transforms_kwargs = {'outer-distance': {'erosion_width': 0}}

7


2022-11-13 00:53:28.991649: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14657 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:04:00.0, compute capability: 7.0


(7, 1040, 1159, 1)


2022-11-13 00:53:28.993063: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14657 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2022-11-13 00:53:28.994275: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14657 MB memory:  -> device: 2, name: Tesla V100-PCIE-16GB, pci bus id: 0000:08:00.0, compute capability: 7.0


(7, 1040, 1159, 1)
2.0
(7, 2080, 2318, 1)
(7, 2080, 2318, 1)
(7, 2080, 2318, 1)
(7, 2080, 2318, 1)
[0, 400, 800, 1200, 1568]
[512, 912, 1312, 1712, 2080]
[0, 512, 1024, 1536, 1806]
[512, 1024, 1536, 2048, 2318]
(175, 512, 512, 1)
(175, 512, 512, 1)


In [4]:
for ii in [1,2,3,4,5]: 
    model_name = 'tn1.0_nuclear_K_512x512resized_train7_aug{}'.format(ii)
    model_path = '{}.h5'.format(model_name)

    # 1. Regular 2. NoScaling 3. NoFlip 4. NoRotation 5. NoFlipRotation 
    if ii==1:
        datagen = image_generators.SemanticDataGenerator(
            rotation_range=180,
            fill_mode='reflect',
            zoom_range=(0.75, 1.25),
            horizontal_flip=True,
            vertical_flip=True)
    elif ii==2:
        datagen = image_generators.SemanticDataGenerator(
            rotation_range=180,
            fill_mode='reflect',
            zoom_range=(1, 1),
            horizontal_flip=True,
            vertical_flip=True)
    elif ii==3:
        datagen = image_generators.SemanticDataGenerator(
            rotation_range=180,
            fill_mode='reflect',
            zoom_range=(0.75, 1.25),
            horizontal_flip=False,
            vertical_flip=False)
    elif ii==4:
        datagen = image_generators.SemanticDataGenerator(
            rotation_range=0,
            fill_mode='reflect',
            zoom_range=(0.75, 1.25),
            horizontal_flip=True,
            vertical_flip=True)
    elif ii==5:
        datagen = image_generators.SemanticDataGenerator(
            rotation_range=0,
            fill_mode='reflect',
            zoom_range=(0.75, 1.25),
            horizontal_flip=False,
            vertical_flip=False)

    # use augmentation for training but not validation
    datagen_val = image_generators.CroppingDataGenerator()

    batch_size = 4 # 8 causes memory outage

    train_data = datagen.flow(
        {'X': sub_X, 'y': sub_y},
        seed=seed,
        transforms=transforms,
        transforms_kwargs=transforms_kwargs,
        min_objects=min_objects,
        batch_size=batch_size)

    val_data = None

    semantic_classes = [1, 1, 2] # inner distance, outer distance, fgbg

    model = PanopticNet(
        backbone='resnet50',
        input_shape=(512,512,1),
        norm_method='whole_image',
        num_semantic_classes=semantic_classes)

    lr = 1e-4
    optimizer = Adam(lr=lr, clipnorm=0.001)
    lr_sched = rate_scheduler(lr=lr, decay=0.99)

    # Create a dictionary of losses for each semantic head

    def semantic_loss(n_classes):
        def _semantic_loss(y_pred, y_true):
            if n_classes > 1:
                return 0.01 * weighted_categorical_crossentropy(
                    y_pred, y_true, n_classes=n_classes)
            return MSE(y_pred, y_true)
        return _semantic_loss

    loss = {}

    # Give losses for all of the semantic heads
    for layer in model.layers:
        if layer.name.startswith('semantic_'):
            n_classes = layer.output_shape[-1]
            loss[layer.name] = semantic_loss(n_classes)

    model.compile(loss=loss, optimizer=optimizer)

    model.load_weights(starting_model_path, by_name=True)

    [(layer.name, layer.output_shape) for layer in filter(lambda x: x.name.startswith('semantic_'), model.layers)]


    # fit the model
    print('Training on', count_gpus(), 'GPUs.')

    train_callbacks = get_callbacks(
        model_path,
        lr_sched=lr_sched,
    #     monitor="val_loss",
        # monitor='loss', # training loss
        verbose=1)          

    loss_history = model.fit(
        train_data,
        steps_per_epoch=train_data.y.shape[0] // batch_size, # 
        epochs=epochs, 
    #     validation_data=val_data,
    #     validation_steps=val_data.y.shape[0] // batch_size,
        callbacks=train_callbacks)

    model.save_weights(model_path)

    prediction_model = PanopticNet(
        backbone='resnet50',
        norm_method='whole_image',
        num_semantic_classes=[1, 1], # inner distance, outer distance
        input_shape= (512,512,1)
    )

    prediction_model.load_weights(model_path, by_name=True)

    from deepcell.applications import NuclearSegmentation
    app = NuclearSegmentation(prediction_model)
    [(layer.name, layer.output_shape) for layer in filter(lambda x: x.name.startswith('semantic_'), app.model.layers)]


    import os
    print(os.getcwd())
    INPUT_PATH="images/test/"
    FILENAMES = [f for f in os.listdir("images/training/testimages")]
    print(FILENAMES)


    # more detailed mapping of image_mpp parameter
    mpps=[1,1.3]

    AP_mat=[]
    for CURR_IM_NAME in FILENAMES:
        print(CURR_IM_NAME)
        im0 = io.imread(os.path.join(INPUT_PATH, CURR_IM_NAME))
        mask_true=io.imread(os.path.join(INPUT_PATH, CURR_IM_NAME.replace("img","masks")))

        x = np.expand_dims(im0, axis=-1)
        x = np.expand_dims(x, axis=0)
        y, tile_info = app._tile_input(x)
        AP_arr=[]
        for mpp in mpps:
            prd = app._untile_output(app.predict(y, image_mpp=mpp), tile_info)[0,:,:,0]
            a=syotil.csi(mask_true, prd)
            AP_arr.append(a)
        AP_mat.append(AP_arr)

    print(AP_mat)

    import pandas as pd
    df = pd.DataFrame([FILENAMES, [x[0] for x in AP_mat], [x[1] for x in AP_mat]])
    print(df.transpose())
    df.to_csv('APresults/csi_'+model_name+'.txt', index=False, header=False)

2022-11-13 00:53:31.840184: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/app/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training on 3 GPUs.


2022-11-13 00:53:38.936310: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 14657 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:04:00.0, compute capability: 7.0
2022-11-13 00:53:38.937400: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:1 with 14657 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2022-11-13 00:53:38.938448: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:2 with 14657 MB memory:  -> device: 2, name: Tesla V100-PCIE-16GB, pci bus id: 0000:08:00.0, compute capability: 7.0


Epoch 1/200


2022-11-13 00:53:54.669830: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8202


...

42/42 [==============================] - 23s 548ms/step - loss: 0.0045 - semantic_0_loss: 0.0014 - semantic_1_loss: 0.0024 - semantic_2_loss: 7.4514e-04 - lr: 1.3808e-05
Epoch 199/200
42/42 [==============================] - ETA: 0s - loss: 0.0047 - semantic_0_loss: 0.0015 - semantic_1_loss: 0.0025 - semantic_2_loss: 7.5352e-04WARNING:tensorflow:Can save best model only with val_loss available, skipping.


42/42 [==============================] - 24s 558ms/step - loss: 0.0047 - semantic_0_loss: 0.0015 - semantic_1_loss: 0.0025 - semantic_2_loss: 7.5352e-04 - lr: 1.3670e-05
Epoch 200/200
42/42 [==============================] - ETA: 0s - loss: 0.0045 - semantic_0_loss: 0.0014 - semantic_1_loss: 0.0024 - semantic_2_loss: 7.3303e-04WARNING:tensorflow:Can save best model only with val_loss available, skipping.


42/42 [==============================] - 23s 549ms/step - loss: 0.0045 - semantic_0_loss: 0.0014 - semantic_1_loss: 0.0024 - semantic_2_loss: 7.3303e-04 - lr: 1.3533e-05
/home/yfong/deeplearning/dense_cell_segmentation
['M926910_CFL_Position7_CD3_test_img.png', 'M872956_JML_Position10_CD3_test_img.png', 'M872956_JML_Position8_CD8_test_img.png', 'M926910_CFL_Position13_CD3_test_img.png', 'M872956_JML_Position8_CD3_test_img.png', 'M872956_JML_Position8_CD4_test_img.png', 'M872956_JML_Position9_CD3_test_img.png']
M926910_CFL_Position7_CD3_test_img.png


/app/software/DeepCell/0.11.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0.Please use `footprint` instead.
  markers = h_maxima(image=maxima,


M872956_JML_Position10_CD3_test_img.png
M872956_JML_Position8_CD8_test_img.png
M926910_CFL_Position13_CD3_test_img.png
M872956_JML_Position8_CD3_test_img.png
M872956_JML_Position8_CD4_test_img.png
M872956_JML_Position9_CD3_test_img.png
[[0.39285714285714285, 0.4550989345509893], [0.36752136752136755, 0.45254237288135596], [0.5025641025641026, 0.48095238095238096], [0.32432432432432434, 0.3684210526315789], [0.5062166962699822, 0.5154109589041096], [0.5640394088669951, 0.5566037735849056], [0.3658051689860835, 0.4437869822485207]]
                                         0         1         2
0   M926910_CFL_Position7_CD3_test_img.png  0.392857  0.455099
1  M872956_JML_Position10_CD3_test_img.png  0.367521  0.452542
2   M872956_JML_Position8_CD8_test_img.png  0.502564  0.480952
3  M926910_CFL_Position13_CD3_test_img.png  0.324324  0.368421
4   M872956_JML_Position8_CD3_test_img.png  0.506217  0.515411
5   M872956_JML_Position8_CD4_test_img.png  0.564039  0.556604
6   M872956_JML_Positio